In [1]:
import time
import re
import pylab as pl
import numpy as np
import pylab as pl

from datetime import datetime, timedelta

import subprocess

import os
import pandas as pd

import concurrent.futures


from joblib import Parallel, delayed
import shutil


from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [2]:
num_nodes = 4

num_txns = 72000
fialed_node_id = 9
write_percent = 10
usecase = 'stack'
operationrate =1000000000

In [3]:
gcp_username = 'tejas'

In [4]:

commands  = []



for i in range(num_nodes):
    cmd =  'gcloud compute instances create tmane-'+f"{i:03}"+' \
    --project=ucr-ursa-major-lesani-lab \
    --zone=us-central1-a \
    --machine-type=e2-highcpu-4 \
    --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default \
    --maintenance-policy=MIGRATE \
    --provisioning-model=STANDARD \
    --service-account=961693926925-compute@developer.gserviceaccount.com \
    --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/trace.append \
    --create-disk=auto-delete=yes,boot=yes,device-name=instance-20241002-130909,image=projects/rhel-cloud/global/images/rhel-8-v20240910,mode=rw,size=20,type=pd-balanced \
    --no-shielded-secure-boot \
    --shielded-vtpm \
    --shielded-integrity-monitoring \
    --labels=goog-ec-src=vm_add-gcloud \
    --reservation-affinity=any'

    commands.append(cmd)


def run_command(command):
    subprocess.call(command, shell=True)


# Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
with concurrent.futures.ThreadPoolExecutor(max_workers=60) as executor:
    # Submit each command to the executor for parallel execution
    futures = [executor.submit(run_command, command) for command in commands]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)

Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-a/instances/tmane-001].
 - The resource 'projects/rhel-cloud/global/images/rhel-8-v20240910' is deprecated. A suggested replacement is 'projects/rhel-cloud/global/images/rhel-8-v20241009'.



NAME       ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
tmane-001  us-central1-a  e2-highcpu-4               10.128.0.16  104.154.22.74  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-a/instances/tmane-002].
 - The resource 'projects/rhel-cloud/global/images/rhel-8-v20240910' is deprecated. A suggested replacement is 'projects/rhel-cloud/global/images/rhel-8-v20241009'.



NAME       ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP  STATUS
tmane-002  us-central1-a  e2-highcpu-4               10.128.0.67  34.71.42.92  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-a/instances/tmane-003].
 - The resource 'projects/rhel-cloud/global/images/rhel-8-v20240910' is deprecated. A suggested replacement is 'projects/rhel-cloud/global/images/rhel-8-v20241009'.



NAME       ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
tmane-003  us-central1-a  e2-highcpu-4               10.128.0.72  35.224.170.69  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-a/instances/tmane-000].
 - The resource 'projects/rhel-cloud/global/images/rhel-8-v20240910' is deprecated. A suggested replacement is 'projects/rhel-cloud/global/images/rhel-8-v20241009'.



NAME       ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
tmane-000  us-central1-a  e2-highcpu-4               10.128.0.73  34.135.232.126  RUNNING


In [5]:
time.sleep(60)

In [6]:
os.system('git add .; git commit -m "test"; git push -f origin HEAD:main')


On branch main
Your branch is behind 'origin/main' by 1 commit, and can be fast-forwarded.
  (use "git pull" to update your local branch)

nothing to commit, working tree clean


To github.com:tejas-shivanand-mane/Optimistic-Replication.git
 + d81c22c...6a4782f HEAD -> main (forced update)


0

In [7]:
def clean_node(i):
    command = (
        f'gcloud compute ssh --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03} '
        f'--project ucr-ursa-major-lesani-lab '
        f'--command "cd; sudo rm -r *"'
    )
    output = subprocess.getoutput(command)
    print(output)
results = Parallel(n_jobs=60, prefer="threads")(delayed(clean_node)(i) for i in range(num_nodes))

rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory
rm: cannot remove '*': No such file or directory


In [ ]:
def git_clone(i):

    command = 'gcloud compute ssh --zone "us-central1-a" "tmane-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Replication.git;"'

    print(command)

    output = os.system(command)
    print(output)
results = Parallel(n_jobs=60, prefer="threads")(delayed(git_clone)(i) for i in range(num_nodes))


gcloud compute ssh --zone "us-central1-a" "tmane-003" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Replication.git;"
gcloud compute ssh --zone "us-central1-a" "tmane-001" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Replication.git;"
gcloud compute ssh --zone "us-central1-a" "tmane-002" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Replication.git;"
gcloud compute ssh --zone "us-central1-a" "tmane-000" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Re

In [ ]:
def kill_code(i):
    command = (
        f'gcloud compute ssh --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03} '
        f'--project ucr-ursa-major-lesani-lab '
        f'--command "sudo killall main-replication-protocols"'
    )
    output = subprocess.getoutput(command)
    print(output)


## RESTART

In [ ]:

for i in range(num_nodes):
    kill_code(i)

    
os.system('git add .; git commit -m "test"; git push -f origin HEAD:main')

def git_pull(i):

    command = 'gcloud compute ssh --zone "us-central1-a" "tmane-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"'

    print(command)

    output = os.system(command)
    print(output)
results = Parallel(n_jobs=60, prefer="threads")(delayed(git_pull)(i) for i in range(num_nodes))

In [ ]:
def setup(i):
    command = (
        f'gcloud compute ssh --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03} '
        f'--project ucr-ursa-major-lesani-lab '
        f'--command "cd Optimistic-Replication; cp optrep_script.sh ../optrep_script.sh; cd ../; sh optrep_script.sh {num_nodes} {num_txns} {write_percent} {usecase} > /home/{gcp_username}/log.txt"'
    )
    os.system(command)

    

In [ ]:
num_txns

In [ ]:
results = Parallel(n_jobs=60, prefer="threads")(delayed(setup)(i) for i in range(num_nodes))

In [ ]:
result = subprocess.check_output('gcloud compute instances list  --format="value(networkInterfaces[0].networkIP)"', shell=True, text=True)

# Convert the output to a list by splitting on newlines
ip_list = result.splitlines()

In [ ]:
ip_list

In [ ]:
def get_sorted_instance_ips():
    result = subprocess.check_output(
        "gcloud compute instances list --filter='name~^tmane-' "
        "--format='value(name,networkInterfaces[0].networkIP)'",
        shell=True
    )
    lines = result.decode().splitlines()
    # Parse into list of tuples: (name, ip)
    name_ip_pairs = [line.strip().split() for line in lines]
    # Sort by name: optrep-000, optrep-001, ...
    sorted_pairs = sorted(name_ip_pairs, key=lambda x: x[0])
    # Extract sorted IPs
    sorted_ips = [ip for name, ip in sorted_pairs]
    return sorted_ips

ip_list = get_sorted_instance_ips()
ip_string = " ".join(ip_list)

In [ ]:
ip_string

In [ ]:
def run_code(i):
    output_dir = f"../../wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/{usecase}"
    output_file = f"{output_dir}/result{i}.out"
    command = (
        f'gcloud compute ssh --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03} '
        f'--project ucr-ursa-major-lesani-lab '
        f'--command "cd Optimistic-Replication/tcp/build; '
        f'sudo bash -c \''
        f'mkdir -p {output_dir}; '
        f'nohup ./main-replication-protocols {i+1} {num_nodes} {num_txns} {write_percent} {usecase} {operationrate} {fialed_node_id} {ip_string} > {output_file} 2>&1 &'
        f'\'"'
    )
    print(command)
    output = subprocess.getoutput(command)
    print(output)



In [ ]:
# results = Parallel(n_jobs=60)(delayed(run_code)(i) for i in range(num_nodes))

for i in range(num_nodes):
    run_code(i)


In [ ]:
time.sleep(30)  # Let it run for 30 seconds
kill_code(3)    # Kill node 3 (id=3, 0-indexed position 2)
time.sleep(60) 

In [ ]:
for i in range(num_nodes):
    kill_code(i)

In [ ]:

if not os.path.exists('../optrep_outputs'):
    os.mkdir('../optrep_outputs')

for i in [0,1, 2, 3]:
    command = (
        f'gcloud compute scp --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03}:/home/{gcp_username}/Optimistic-Replication/wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/stack/result{i}.out '
        f'../optrep_outputs/{i}.txt '
        f'--project ucr-ursa-major-lesani-lab'
    )
    os.system(command)
    # break

In [ ]:
# os.system('gcloud compute instances delete --zone=us-central1-a --quiet --project=ucr-ursa-major-lesani-lab $(gcloud compute instances list --project=ucr-ursa-major-lesani-lab --filter="name~^tmane-" --format="value(name)")')